In [1]:
import sys

sys.path.append("C:\\Users\\micha\\work\\Meal-Master")

In [2]:
from structure import *
from database import myDB

In [3]:
import mysql.connector
host = "localhost"
user = "root"
password = "820510"

conn = mysql.connector.connect(
  host=host,
  user=user,
  password=password
)

In [4]:
db = myDB(conn)

In [5]:
db.create_tables()

DatabaseError: 1824 (HY000): Failed to open the referenced table 'city'

In [4]:
class Recommender:
  def __init__(self, res_db, per_db):
    self.res_db = res_db
    self.per_db = per_db

  def filter_by_options(restaurants):
    return self.res_db.get_restaurant({'cuisine': cuisine, 'price': price, 'location': location, 'env': env})
  
  def sort_by_preference(user):
    self.per_db.get
  
  def get_similar_users(user, k):
    # return the closest k users

    def get_score(per):
      return 1 # consine similarity of per and user

    out = [] # (score, i)
    for i, per in enumerate(self.per_db):
      user_id = per._id
      score = self.get_score(per)

      if len(out) < k:
        out.append((score, user_id))
      else:
        if out[0][0] < score:
          out.pop(0)
          out.append((score, user_id))
      out.sort(key = lambda p: p[0])
    return out


  def get_recommendations(user: User, topk = 10):
    # Sort the restaurants by rating in descending order
    neis = get_similar_users(user, k=3)
    history = []
    for nei in neis:
      history.append(nei.history)

    # user mapReduce to get distict restaurant and its average rating
    res = mapReduce(history)
    res = self.filter_by_options(res)

    if len(res) < topk:
      other_res = self.filter_by_options(self.res_db)
      res + other_res[:topk-len(res)]
      
    return res